# Generation of reservoir inflow ensemble forecasts from weather forecasts
In this Notebook we are going to generate reservoir inflow ensemble forecasts using seasonal meteorological forecasts produced by a dynamic climate model. In this case we will use the European Centre for Medium-Range Weather Forecast (ECMWF) seasonal meteorological forecast that will feed the HBV rainfall-runoff model to simulate the dynamic streamflow predictions (DSP).

To run this Notebook we need to import some necessary libraries. **Only if iRONs is run locally**: since one required library, [plotly](https://plot.ly/), is not available on Anaconda by default, you must have installed it first. Help on how to install libraries is given here: [How to install libraries](../0%20-%20Tutorials/0.b%20-%20How%20to%20install%20libraries.ipynb). If iRONs is run on the cloud, e.g. on [Binder](https://mybinder.org/) or [Microsoft Azure Notebooks](https://notebooks.azure.com/), we do not need to install the libraries to import them. 

Once all the necessary libraries are installed locally or in case we are running iRONs on the cloud, we import the libraries with the following code:

In [12]:
import numpy as np
from datetime import datetime, timedelta
import ipywidgets as widgets
import plotly.graph_objs as go
import os
# Sub-modules
from Submodules.date_num import num2date, date2num
from Submodules.HBV_model import HBV_model

## 1. Forcing inputs
In this example the reservoir inflow forecast simulation is forced by the bias corrected ECMWF seasonal weather forecast.
We use seasonal weather forecast data, previously downloaded and bias corrected to simulate the reservoir inflow forecast for the next 7 months.

To load the data we first set the forecast product (ECMWF as forecast orginitating centre in this case).

In [3]:
origin_centre = 'ECMWF' # forecast originating centre
folder_path = '../1 - Seasonal weather forecast/Results/'+origin_centre # folder containing the forecast weather data
DSP_E_file = folder_path+'/Fore_Evap.csv'
DSP_P_file = folder_path+'/Fore_Rain.csv'

skip_lines = 1 # The number of lines to skip at the beginning of the .csv file.
date_format = "%d/%m/%Y" # Format of the dates on the .csv files
dates_DSP = np.genfromtxt(DSP_E_file, 
                      delimiter = ',',
                      skip_header=skip_lines,
                      dtype = object,
                      converters={0: lambda x: datetime.strptime(x.decode("utf-8"), "%d/%m/%Y")})[0:,0]
E_DSP = np.genfromtxt(DSP_E_file, 
                    delimiter = ',',
                    skip_header=skip_lines)[0:,1:] # in mm/day
P_DSP = np.genfromtxt(DSP_P_file, 
                    delimiter = ',',
                    skip_header=skip_lines)[0:,1:] # in mm/day

Plotting the precipitation forecast ensemble

In [1]:
# First we define the traces (layers of data)
num_mem = np.shape(P_DSP)[1]
trace_E_DSP = [0]*num_mem
trace_P_DSP = [0]*num_mem
for m in range(num_mem):
    trace_E_DSP[m] = go.Scatter(x = dates_DSP,
                                y = E_DSP[:,m],
                                name = "member "+str(m),
                                line=dict(color='yellow', width=1),
                                opacity=0.5)
    trace_P_DSP[m] = go.Scatter(x = dates_DSP,
                                y = P_DSP[:,m],
                                name = "member "+str(m),
                                line=dict(color='blue', width=1),
                                opacity=0.5)

# Then the figure layout
layout1 = go.Layout(title="Forecast ensemble - Daily precipitation",
                    xaxis=dict(showgrid=False,title = 'date'),
                    yaxis=dict(title = 'mm/day'),
                    width=900, 
                    height=400)
# Finally we plot figure using the pre-defined layout and traces
fig1 = go.Figure(data=trace_P_DSP,
               layout = layout1)
fig1

NameError: name 'P_DSP' is not defined

## 2. Model parameters

In [8]:
BETA   = 6.4    # exponential parameter in soil routine [-]
LP     = 0.8    # evapotranspiration limit [-]
FC     = max(np.finfo(float).eps,247.1)  # field capacity [mm] 
PERC   = 0.9    # maximum flux from Upper to Lower Zone [mm/Dt]
K0     = 1/3.9  # near surface flow coefficient (ratio) [1/Dt]  
K1     = 1/11.0 # upper Zone outflow coefficient (ratio) [1/Dt]  
K2     = 1/61.3 # lower Zone outflow coefficient (ratio) [1/Dt]  
UZL    = 27.3   # near surface flow threshold [mm]
MAXBAS = max(1,round(0.9))    # flow routing coefficient or Transfer function parameter [day]

param = [BETA, LP, FC, PERC, K0, K1, K2, UZL, MAXBAS]

case = 1 # preferred path in the Upper Zone dynamics

catchment_area = 28.8 # km2

## 3. Warm-up
We use historical weather data (precipitation and PET) to simulate the historical reservoir inflow from 1980. 

If observed weather data is not available in our area of study, in the UK we can download historical daily climate data from the [CEH website](https://eip.ceh.ac.uk/apps/chess/). First we need to find your study are in the map and click on this location. Then we need to set the type of weather data we would like to download and the time period (the maximum length is 12 months). 

In this example, we have a single CSV file (Hist_clim_data.csv) with observed daily PET [mm], precipitation [mm] and mean temperature [degC] from 1980 to 2015.

In [5]:
Hist_data_file = 'Input data/Historical data/Hist_clim_data.csv'

skip_lines = 1 # The number of lines to skip at the beginning of the .csv file.
date_format = "%d/%m/%Y" # Format of the dates on the .csv files
dates_hist = np.genfromtxt(Hist_data_file, 
                      delimiter = ',',
                      skip_header=skip_lines,
                      dtype = object,
                      converters={0: lambda x: datetime.strptime(x.decode("utf-8"), "%d/%m/%Y")})[0:,0]
E_hist = np.genfromtxt(Hist_data_file, 
                    delimiter = ',',
                    skip_header=skip_lines)[0:,1] # in mm/day
P_hist = np.genfromtxt(Hist_data_file, 
                    delimiter = ',',
                    skip_header=skip_lines)[0:,2] # in mm/day 

Initial conditions

In [6]:
# We first set arbitrary values for the initial condtions
SSM0 = 200 # initial soil moisture [mm]
SUZ0 = 2.5 # initial Upper Zone storage [mm]
SLZ0 = 2.5 # initial Lower Zone storage [mm]
ini0 = [SSM0,SUZ0,SLZ0]
# We extract the weather data for the warm-up period
indices_warmup = np.arange(0,(dates_DSP[0]-dates_hist[0]).days)
dates_warmup= dates_hist[indices_warmup]
E_warmup = E_hist[indices_warmup]
P_warmup = P_hist[indices_warmup]
# Then we run the model with the arbritrarly defined initial conditions and the calibrated parameters
[SM_warmup,UZ_warmup,LZ_warmup], [EA,R,RL,Q0,Q1,Q] = HBV_model(dates_warmup, E_warmup, P_warmup, ini0, param, case)
ini_DSP = [SM_warmup[-1],UZ_warmup[-1],LZ_warmup[-1]]

## 4. Simulation

In [9]:
I_DSP = np.zeros(np.shape(P_DSP))
trace_I_DSP = [0]*num_mem
for m in range(num_mem):
    [SM,UZ,LZ], [EA,R,RL,Q0,Q1,Q_DSP] = HBV_model(dates_DSP, E_DSP[:,m], P_DSP[:,m], ini_DSP, param, case)
    I_DSP[:,m] = Q_DSP*catchment_area # flows Q (in mm) * area (km2) = I (in ML)
    # we define the traces (layers of data to plot)
    trace_I_DSP[m] = go.Scatter(x = dates_DSP,
                                y = I_DSP[:,m],
                                name = "member "+str(m),
                                line=dict(color='blue', width=1),
                                opacity=0.5)

Plotting the DSP inflow forecast

In [10]:
# Then the figure layout
layout4 = go.Layout(title="DSP forecast ensemble - Daily inflow",
                    xaxis=dict(showgrid=False,title = 'date'),
                    yaxis=dict(title = 'ML/day'),
                    width=900, 
                    height=400)
# Finally we plot figure using the pre-defined layout and traces
fig4 = go.Figure(data=trace_I_DSP,
               layout = layout4)
fig4

## 5. Transform into weekly cumulative inflows

In [67]:
# First we define the forecast horizon in number of weeks
horizon = 28 # weeks
# Initial day
date0 = dates_DSP[0]
# Day of the week of the initial day (Monday = 0,..., Sunday = 6)
wday0 = date0.weekday()
# We define the inital date according to the day of the week we would like to start with, in this case Monday
if wday0 != 0:
    date_ini = date0 + timedelta(days = 7-wday0)
# Given the initial date and the forecast horizon we now get the final date
date_end = date_ini + timedelta(days = horizon*7) # day_ini + horizon weeks * 7 days/week
if (date_end-dates_DSP[-1]).days > 0:
    print('Error: The defined horizon is too long, please try with a lower number of weeks')

In [79]:
index_ini = np.where(dates_DSP==date_ini)[0][0]
count = index_ini
dates_DSP_weekly = dates_DSP[index_ini]
I_DSP_cum_weekly = [np.zeros(np.shape(I_DSP)[1])]
delta = 7 # days of a week
for i in np.arange(horizon)+1:
    dates_DSP_weekly = np.append(dates_DSP_weekly,[dates_DSP[index_ini+i*delta]])
    I_DSP_cum_weekly = np.append(I_DSP_cum_weekly,[np.sum(I_DSP[index_ini:index_ini+i*delta,:],axis =0)],axis = 0)

Plotting the DSP inflow forecast

In [85]:
trace_I_DSP_cum_weekly = [0]*num_mem
for m in range(num_mem):
    trace_I_DSP_cum_weekly[m] = go.Scatter(x = dates_DSP_weekly,
                                y = I_DSP_cum_weekly[:,m],
                                name = "member "+str(m),
                                line=dict(color='blue', width=1),
                                opacity=0.5)
# Then the figure layout
layout5 = go.Layout(title="DSP forecast ensemble - Weekly cumulative inflow",
                    xaxis=dict(showgrid=False,title = 'date'),
                    yaxis=dict(title = 'ML'),
                    width=900, 
                    height=400)
# Finally we plot figure using the pre-defined layout and traces
fig5 = go.Figure(data=trace_I_DSP_cum_weekly,
               layout = layout5)
fig5

## 6. Save the resutls
Save the historical inflow simulation into a csv file

In [98]:
# First we need to transform the data that we want to save into strings
dates_DSP_str = np.reshape([dates_DSP_weekly[i].strftime("%d/%m/%Y") for i in range(len(dates_DSP_weekly))],
                           (len(dates_DSP_weekly),1))
I_DSP_str = np.zeros(I_DSP_cum_weekly.shape)
num_row = I_DSP_cum_weekly.shape[0]
num_col = I_DSP_cum_weekly.shape[1]
for j in np.arange(num_col):
    for i in np.arange(num_row):
        I_DSP_str[i,j] = str(I_DSP_cum_weekly[i,j])
# Then we can save the data in string format in a csv file
if not os.path.exists('Results'):
    os.mkdir('Results')
np.savetxt('Results/DSP_weekly_cum_inflow.csv',
           np.append(dates_DSP_str,I_DSP_str, axis=1), 
           fmt = '%s', 
           delimiter=',',
           header = 'cumulative inflow in ML')

### References 

Bergström, S. (1992) The HBV model - its structure and applications. SMHI Reports RH, No. 4, Norrköping.